In [1]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv(r"Data/Raw/game.csv")

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34163 entries, 31535 to 65697
Data columns (total 55 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   season_id               34163 non-null  int64         
 1   team_id_home            34163 non-null  int64         
 2   team_abbreviation_home  34163 non-null  object        
 3   team_name_home          34163 non-null  object        
 4   game_id                 34163 non-null  int64         
 5   game_date               34163 non-null  datetime64[ns]
 6   matchup_home            34163 non-null  object        
 7   wl_home                 34161 non-null  object        
 8   min                     34163 non-null  int64         
 9   fgm_home                34163 non-null  float64       
 10  fga_home                34163 non-null  float64       
 11  fg_pct_home             34163 non-null  float64       
 12  fg3m_home               34163 non-null  float64

In [ ]:
df.isna().sum()

season_id                     0
team_id_home                  0
team_abbreviation_home        0
team_name_home                0
game_id                       0
game_date                     0
matchup_home                  0
wl_home                       2
min                           0
fgm_home                     13
fga_home                  15447
fg_pct_home               15490
fg3m_home                 13218
fg3a_home                 18683
fg3_pct_home              19074
ftm_home                     16
fta_home                   3004
ft_pct_home                3009
oreb_home                 18936
dreb_home                 18999
reb_home                  15729
ast_home                  15805
stl_home                  18849
blk_home                  18626
tov_home                  18684
pf_home                    2856
pts_home                      0
plus_minus_home               0
video_available_home          0
team_id_away                  0
team_abbreviation_away        0
team_nam

In [5]:
df.duplicated().sum()

np.int64(0)

FASE DE CONVERSIÓN DE FEHCA Y FILTRADO HISTÓRICO

In [6]:
# 1. Conversión de game_date a datetime
df['game_date'] = pd.to_datetime(df['game_date'])

# 2. Filtrado por fehca
# # La fecha de corte se establece después del 31 de octubre de 1996 (ya que la temporada comienza a finales de octubre)
df = df[df['game_date'] > '1996-10-31'].copy()

print(f"Filas después del filtrado: {len(df)}")


Filas después del filtrado: 34163


CABIAR TIPO DE DATOS IDs a INT

In [ ]:
# Conversión de IDs a int
id_cols = [
    "season_id", "team_id_home", "team_id_away",
    "game_id"
]

for col in id_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int64")


ELIMINACIÓN DE COLUMNAS REDUNDANTES

In [ ]:
# Identificar y eliminar columnas redundantes
# Tabmién se eliminarán las columnas de porcentaje, ya que si contienen valores nulos conviene recarcularlas luego en el análisis exploratorio

cols_redundantes = [
    "video_available_home",
    "video_available_away",
    "team_abbreviation_home",
    "team_abbreviation_away",
    "team_name_home",
    "team_name_away",
    "matchup_home",
    "matchup_away",
    "fg_pct_home",
    "fg3_pct_home",
    "ft_pct_home",
    "fg_pct_away",
    "fg3_pct_away",
    "ft_pct_away"
]

df = df.drop(columns=[c for c in cols_redundantes if c in df.columns])


MANEJO DE VALORES NULOS

In [ ]:
# 1. Nulos en métricas home/away
# Rellenar con 0 aquellas columnas que no sean calculadas (porcentajes), estas pueden calcularse posteriormente
metricas_con_ceros = [
    "fgm_home", "fga_home", "fg3m_home", "fg3a_home", "ftm_home", "fta_home",
    "oreb_home", "dreb_home", "reb_home",
    "ast_home", "stl_home", "blk_home", "tov_home", "pf_home", "pts_home",
    
    "fgm_away", "fga_away", "fg3m_away", "fg3a_away", "ftm_away", "fta_away",
    "oreb_away", "dreb_away", "reb_away",
    "ast_away", "stl_away", "blk_away", "tov_away", "pf_away", "pts_away"
]

for col in metricas_con_ceros:
    if col in df.columns:
        df[col] = df[col].fillna(0).astype("Int64")

In [22]:
# 2. Eliminación de filas con nulos críticos (W/L)
# Es una variable categórica binaria que no se puede imputar con una media o un cero. Al ser solo 2 filas del dataset, el impacto no es significativo
df.dropna(subset=['wl_home', 'wl_away'], inplace=True)

VALIDACIONES Y CORRECCIONES

In [23]:
# Reemplazar valores que carecen de sentido (en el caso de que eistan)
# Por ejemplo: partidos con pts = 0 para ambos equipos representan errores
df = df.loc[~((df["pts_home"] == 0) & (df["pts_away"] == 0))]

# Eliminar duplicados
df = df.drop_duplicates(subset=["game_id"])


VERIFICAR TRANSFORMACIÓN DE DATOS

In [24]:
df.head()

,season_id,team_id_home,game_id,game_date,wl_home,min,fgm_home,fga_home,fg3m_home,fg3a_home,...,dreb_away,reb_away,ast_away,stl_away,blk_away,tov_away,pf_away,pts_away,plus_minus_away,season_type
31535,21996,1610612747,29600012,1996-11-01,W,240,31,63,6,18,...,21,36,21,11,4,12,25,82,-14,Regular Season
31536,21996,1610612748,29600005,1996-11-01,W,240,35,78,10,21,...,30,40,13,10,7,24,20,81,-13,Regular Season
31537,21996,1610612751,29600002,1996-11-01,L,240,23,58,9,18,...,23,35,16,10,1,15,24,90,13,Regular Season
31538,21996,1610612765,29600007,1996-11-01,W,240,32,68,6,20,...,26,36,19,6,4,18,30,89,-6,Regular Season
31539,21996,1610612744,29600013,1996-11-01,L,240,27,75,7,18,...,33,49,21,7,2,20,27,97,12,Regular Season


In [25]:
df.isnull().sum()

season_id          0
team_id_home       0
game_id            0
game_date          0
wl_home            0
min                0
fgm_home           0
fga_home           0
fg3m_home          0
fg3a_home          0
ftm_home           0
fta_home           0
oreb_home          0
dreb_home          0
reb_home           0
ast_home           0
stl_home           0
blk_home           0
tov_home           0
pf_home            0
pts_home           0
plus_minus_home    0
team_id_away       0
wl_away            0
fgm_away           0
fga_away           0
fg3m_away          0
fg3a_away          0
ftm_away           0
fta_away           0
oreb_away          0
dreb_away          0
reb_away           0
ast_away           0
stl_away           0
blk_away           0
tov_away           0
pf_away            0
pts_away           0
plus_minus_away    0
season_type        0
dtype: int64

EXPORTAR ARCHIVO LIMPIO

In [27]:
# Esportar archivo
df.to_csv(r"Data/Processed/game_limpio.csv", index=False)